In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sklearn
import time
import re
from datetime import date as Date
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [2]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_UFC_events")
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
table = soup.find("table", id="Past_events")
allEventsLinks = table.find("tbody").find_all("tr")


In [5]:

def formatDate(dateToFormat):
    splitDate = ""
    
    if dateToFormat.find(" ") != -1:
        splitData = dateToFormat.split(" ")
        
        if len(splitData) < 3:
            return None
        
        if splitData[0].isdigit() == True:
            splitDate = splitData[1] + splitData[0] + "," + splitData[2]
        else:
            splitDate = splitData[0] + splitData[1] + "," + splitData[2]
            
        return splitDate
            
            
            
    if dateToFormat[0].isdigit() == False:
        return dateToFormat
    
    splitDate = dateToFormat.split(",")
    
    
    if len(splitDate) < 2:
        #print(dateToFormat)
        return None
    
    leftSide = splitDate[0]
    if len(leftSide) < 2:
        return None
    
    #May27,2022 -> currect one.
    #27May,2022
    for i in range(2):
        
        if leftSide[i].isdigit():
            temp = leftSide[ i + 1:]
            temp += leftSide[i]
            leftSize = temp
        else:
            return str(leftSize + splitDate[1])
        
    return None
            
        

In [6]:
# get fighter wins and loses
def getFighterWinsAndLoses(link, date, isWinner):
    # get fighter wins and loses
    fighterPage = requests.get(link)
    fighterSoup = BeautifulSoup(fighterPage.content, 'html.parser')
    
    infoDiv = fighterSoup.find("div", class_="mw-parser-output").find("p");
    
    table = fighterSoup.find_all("table", class_="wikitable")
    
    #fix when the fighter page got only 1 info table (need to be 2 tables)
    if len(table) < 1:
        print("there are no table. fighter: " + link)
        print("date: " + date)
        print("isWinner: " + str(isWinner))
        return None;
    
    elif len(table) < 2:
        print("ONE table only. fighter: " + link)
        print("date: " + date)
        print("isWinner: " + str(isWinner))
        table = table[0]
    elif len(table) == 2:
        if len(table[0].find_all("tr")) > len(table[1].find_all("tr")):
            table = table[0]
        else:
            table = table[1]
    else:
        if len(table[1].find_all("tr")) > len(table[2].find_all("tr")):
            table = table[1]
        else:
            table = table[2]
            
    allTr = table.find_all("tr")
    
    for data in allTr[1:]:
        allTd = data.find_all("td")
        if len(allTd) < 6:
            print("fighter data rows is lower then 6.   fighter: " + link)
            print(allTd)
            continue
                        
        currInfoDate = allTd[5].text.strip()
        
        #remove spaces in strings to compare
        date = re.sub(r'\s+', '', date)
        
        if currInfoDate.find(",") != -1:
            currInfoDate = re.sub(r'\s+', '', currInfoDate)
        
        #check if curr info date is starts with a number. then formatting the string to starts with date/day/year
        if currInfoDate == None or len(currInfoDate) == 0:
            continue
            
        if currInfoDate[0].isdigit():
            currInfoDate = formatDate(currInfoDate);
            if currInfoDate == None:
                continue
                
        
        if date.strip() == currInfoDate.strip():
            scoreData = str(data.find_all("td")[1].text.strip().split("(")[0].strip())
            score = scoreData.split("–")
            
            if len(score) < 2:
                score = score[0].split("-")
                if len(score) < 2:
                    print("score split size is not correct. fighter: " + link)
                    print(score)
                    
                    return None;
            
            wins = score[0].strip()
            loses = score[1].strip()
            
            
            # fixing wins and loses to be as before the fight and not after
            
            if isWinner:
                wins = int(wins) - 1
            else:
                loses = int(loses) - 1
            
            return (wins, loses)
    
    print("fighter data not found.   fighter: " + link)
    print("date: " + date)
    print("isWinner: " + str(isWinner))
    
    return None;

In [7]:
#get fighter age
def getFighterData(name):
    fighterName = name.replace(" ", "-")
    link = "https://www.ufc.com/athlete/" + fighterName
    
    fighterPage = requests.get(link)
    fighterSoup = BeautifulSoup(fighterPage.content, 'html.parser')
    
    age = 0
    height = 0
    reach = 0
    legReach = 0
    

    if fighterSoup.find("div", class_="c-bio__info-details") != None:
        bioDataRows = fighterSoup.find("div", class_="c-bio__info-details").find_all("div", class_="c-bio__row--3col")
        bioData1 = bioDataRows[0].find_all("div", class_="c-bio__field")
        bioData2 = bioDataRows[1].find_all("div", class_="c-bio__field")
        
        
        age = bioData1[0].find("div", class_="field")
        if age != None:
            age = age.text.strip()
            
        height = bioData1[1].find("div", class_="c-bio__text")
        if height != None:
            height = height.text.strip()
        
        if len(bioData2) > 1:
            reach = bioData2[1].find("div", class_="c-bio__text").text.strip()
        
        if len(bioData2) > 2:
            legReach = bioData2[2].find("div", class_="c-bio__text").text.strip()
        
    
    return {"age": age, "height": height, "reach": reach, "legReach": legReach}

In [ ]:
#get all fight event links

winners = []
losers = []
weightClasses = []
ages = []
methods = []
winRounds = []
winTimes = []
eventNames = []
dates = []

winnerHeights = []
loserHeights = []
winnerReachs = []
loserReachs = []
winnerLegReachs = []
loserLegReachs = []

winnerAges = []
loserAges = []
winnerWins = []
winnerLoses = []
loserWins = []
loserLoses = []

print(len(allEventsLinks))

startTime = time.perf_counter()
#for idx in range(1, len(allEventsLinks)):
for idx in range(1, len(allEventsLinks)):
    eventPage = requests.get("https://en.wikipedia.org" + allEventsLinks[idx].find("a")['href'])
    eventSoup = BeautifulSoup(eventPage.content, 'html.parser')
    
    print("---------- " + str(idx) + " ------------")
    
    table = eventSoup.find("table", class_="toccolours")
    
    if table == None:
        continue
        
        
    allTr = table.find_all("tr")
    eventName = eventSoup.find(id="firstHeading").text.strip()
    date = ""
    
    # get event date
    infoTable = eventSoup.find("table", class_="infobox").find_all("tr")
    for info in infoTable:
        if info.find("th") != None and info.find("th").text.strip() == "Date":
            date = info.find("td").text.strip().split("(")[0].strip()
            break
    
    
    # get fight details
    for fightIdx in range(1, len(allTr)):
        curr = allTr[fightIdx].find_all("td")
        
        if curr == []:
            continue
            
        # get fighter wins and loses
        if curr[1].find("a") == None or curr[3].find("a") == None:
            continue
            
        winnerName = curr[1].text.strip()
        loserName = curr[3].text.strip()
        
        winnerLink = "https://en.wikipedia.org/wiki/" + curr[1].find("a")['href'].strip().split("/")[2]
        loserLink = "https://en.wikipedia.org/wiki/" + curr[3].find("a")['href'].strip().split("/")[2]

        
        winnerWinsAndLoses = getFighterWinsAndLoses(winnerLink, date, True)
        loserWinsAndLoses = getFighterWinsAndLoses(loserLink, date, False)
        
        if winnerWinsAndLoses == None or loserWinsAndLoses == None:
            continue
        
        eventYear = int(date[len(date) - 5:])
        
        winnerData = getFighterData(winnerName)
        loserData = getFighterData(loserName)
        
        #fix age with event year
        if winnerData['age'] != None:
            winnerAge = (eventYear - Date.today().year) + int(winnerData['age'])
            
        if loserData['age'] != None:
            loserAge = (eventYear - Date.today().year) + int(loserData['age'])
        
        

        weightClasses.append(curr[0].text.strip())
        winners.append(curr[1].text.strip())
        losers.append(curr[3].text.strip())
        methods.append(curr[4].text.strip())
        winRounds.append(curr[5].text.strip())
        winTimes.append(curr[6].text.strip())
        
        winnerHeights.append(winnerData['height'])
        winnerReachs.append(winnerData['reach'])
        winnerLegReachs.append(winnerData['legReach'])
        
        loserHeights.append(loserData['height'])
        loserReachs.append(loserData['reach'])
        loserLegReachs.append(loserData['legReach'])
        
        
        winnerAges.append(winnerAge);
        loserAges.append(loserAge);
        winnerWins.append(winnerWinsAndLoses[0])
        winnerLoses.append(winnerWinsAndLoses[1])
        loserWins.append(loserWinsAndLoses[0])
        loserLoses.append(loserWinsAndLoses[1])
        
        eventNames.append(eventName)
        dates.append(date)
        
endTime = time.perf_counter()

print(f"time took: {endTime - startTime:0.4f} seconds")

620
---------- 1 ------------
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Carlos_Ulberg
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Carlos_Ulberg
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Carlos_Ulberg
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Carlos_Ulberg
[<td colspan="9"><i><b>Legend</b></i>: <style data-mw-deduplicate="TemplateStyles:r981673959">.mw-parser-output .legend{page-break-inside:avoid;break-inside:avoid-column}.mw-parser-output .legend-color{display:inline-block;min-width:1.25em;height:1.25em;line-height:1.25;margin:1px 0;text-align:center;border:1px solid black;background-color:transparent;color:black}.mw-parser-output .legend-text{}</style><span class="legend nowrap"><span class="legend-color" style="border: 1px solid #CCFFCC;background-color:#CCFFCC; color:black;"> </span> Win </span> <link href="mw-data:TemplateStyles:r981673959"

---------- 3 ------------
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Glover_Teixeira
date: June12,2022
isWinner: False
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Zhang_Weili
date: June12,2022
isWinner: True
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Joanna_J%C4%99drzejczyk
date: June12,2022
isWinner: False
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Jake_Matthews_(fighter)
date: June12,2022
isWinner: True
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Andr%C3%A9_Fialho
date: June12,2022
isWinner: False
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Joshua_Culibao
date: June12,2022
isWinner: True
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Seung_Woo_Choi
date: June12,2022
isWinner: False
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Brendan_Allen
date: June12,2022
isWinner: True
fighter data not found.   fighter: https://en.wi

---------- 19 ------------
---------- 20 ------------
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Ciryl_Gane
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Ciryl_Gane
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Ciryl_Gane
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Ciryl_Gane
[<td colspan="9"><i><b>Legend</b></i>: <style data-mw-deduplicate="TemplateStyles:r981673959">.mw-parser-output .legend{page-break-inside:avoid;break-inside:avoid-column}.mw-parser-output .legend-color{display:inline-block;min-width:1.25em;height:1.25em;line-height:1.25;margin:1px 0;text-align:center;border:1px solid black;background-color:transparent;color:black}.mw-parser-output .legend-text{}</style><span class="legend nowrap"><span class="legend-color" style="border: 1px solid #CCFFCC;background-color:#CCFFCC; color:black;"> </span> Win </span> <link href="mw-data:TemplateStyles

fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Brad_Riddell
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Brad_Riddell
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Brad_Riddell
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Brad_Riddell
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Brad_Riddell
[<td colspan="9"><i><b>Legend</b></i>: <style data-mw-deduplicate="TemplateStyles:r981673959">.mw-parser-output .legend{page-break-inside:avoid;break-inside:avoid-column}.mw-parser-output .legend-color{display:inline-block;min-width:1.25em;height:1.25em;line-height:1.25;margin:1px 0;text-align:center;border:1px solid black;background-color:transparent;color:black}.mw-parser-output .legend-text{}</style><span class="legend nowrap"><span class="legend-color" style="border: 1px solid #CCFFCC;background-color:#CCFFCC; color:black;"> </span> 

fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Giga_Chikadze
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Giga_Chikadze
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Giga_Chikadze
[<td colspan="9"><i><b>Legend</b></i>: <style data-mw-deduplicate="TemplateStyles:r981673959">.mw-parser-output .legend{page-break-inside:avoid;break-inside:avoid-column}.mw-parser-output .legend-color{display:inline-block;min-width:1.25em;height:1.25em;line-height:1.25;margin:1px 0;text-align:center;border:1px solid black;background-color:transparent;color:black}.mw-parser-output .legend-text{}</style><span class="legend nowrap"><span class="legend-color" style="border: 1px solid #CCFFCC;background-color:#CCFFCC; color:black;"> </span> Win </span> <link href="mw-data:TemplateStyles:r981673959" rel="mw-deduplicated-inline-style"/><span class="legend nowrap"><span class="legend-color" style="border: 1px solid #FFBBB

...

---------- 163 ------------
---------- 164 ------------
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Conor_McGregor
[<td><a href="/wiki/Sky_Box_Office" title="Sky Box Office">Sky Box Office</a> (UK)
</td>, <td>1,007,000<sup class="reference" id="cite_ref-288"><a href="#cite_note-288">[288]</a></sup>
</td>]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Conor_McGregor
[]
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Conor_McGregor
date: October6,2018
isWinner: False
---------- 165 ------------
---------- 166 ------------
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Mark_Hunt
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Mark_Hunt
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Mark_Hunt
[]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Mark_Hunt
[]
fighter data rows is lower then 6.   fi

fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Cub_Swanson
[<td>2012</td>, <td><i><a href="/wiki/UFC_Undisputed_3" title="UFC Undisputed 3">UFC Undisputed 3</a></i></td>, <td>Himself</td>, <td>Playable character
</td>]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Cub_Swanson
[<td>2014</td>, <td><i><a href="/wiki/EA_Sports_UFC" title="EA Sports UFC">EA Sports UFC</a></i></td>, <td>Himself</td>, <td>Playable character
</td>]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Cub_Swanson
[<td>2016</td>, <td><i><a href="/wiki/EA_Sports_UFC_2" title="EA Sports UFC 2">EA Sports UFC 2</a></i></td>, <td>Himself</td>, <td>Playable character
</td>]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Cub_Swanson
[<td>2018</td>, <td><i><a href="/wiki/EA_Sports_UFC_3" title="EA Sports UFC 3">EA Sports UFC 3</a></i></td>, <td>Himself</td>, <td>Playable character
</td>]
fighter data rows is lo

---------- 174 ------------
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Israel_Adesanya
date: 
isWinner: True
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Brad_Tavares
date: 
isWinner: False
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Alex_Caceres
date: 
isWinner: True
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Martin_Bravo
date: 
isWinner: False
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Roxanne_Modafferi
date: 
isWinner: True
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Barb_Honchak
date: 
isWinner: False
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Alessio_Di_Chirico
date: 
isWinner: True
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Julian_Marquez
date: 
isWinner: False
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Montana_De_La_Rosa
date: 
isWinner: True
fighter data not found.   fighter: https://en.

fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/CM_Punk
[<td>2004
</td>, <td><i><a href="/wiki/Monster_Garage" title="Monster Garage">Monster Garage</a></i>
</td>, <td>Himself
</td>, <td>Episode: "Box Truck Wrestling Car"
</td>, <td align="center"><sup class="reference" id="cite_ref-364"><a href="#cite_note-364">[364]</a></sup>
</td>]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/CM_Punk
[<td>2006
</td>, <td><i><a href="/wiki/Ghost_Hunters_(TV_series)" title="Ghost Hunters (TV series)">Ghost Hunters</a></i>
</td>, <td>Himself
</td>, <td>Episode: "Live Halloween Special: Stanley Hotel"
</td>, <td align="center"><sup class="reference" id="cite_ref-365"><a href="#cite_note-365">[365]</a></sup>
</td>]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/CM_Punk
[<td>2011
</td>, <td><i><a href="/wiki/Jimmy_Kimmel_Live!" title="Jimmy Kimmel Live!">Jimmy Kimmel Live!</a></i>
</td>, <td>Himself
</td>, <td>1 episod

---------- 178 ------------
---------- 179 ------------
---------- 180 ------------
fighter data not found.   fighter: https://en.wikipedia.org/wiki/Mackenzie_Dern
date: May12,2018
isWinner: True
---------- 181 ------------
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Cub_Swanson
[<td>2012</td>, <td><i><a href="/wiki/UFC_Undisputed_3" title="UFC Undisputed 3">UFC Undisputed 3</a></i></td>, <td>Himself</td>, <td>Playable character
</td>]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Cub_Swanson
[<td>2014</td>, <td><i><a href="/wiki/EA_Sports_UFC" title="EA Sports UFC">EA Sports UFC</a></i></td>, <td>Himself</td>, <td>Playable character
</td>]
fighter data rows is lower then 6.   fighter: https://en.wikipedia.org/wiki/Cub_Swanson
[<td>2016</td>, <td><i><a href="/wiki/EA_Sports_UFC_2" title="EA Sports UFC 2">EA Sports UFC 2</a></i></td>, <td>Himself</td>, <td>Playable character
</td>]
fighter data rows is lower then 6.   fighter

In [ ]:
# creating the Dataframe
dfData = {
    'winner': winners,
    'loser': losers,
    'weight class': weightClasses,
    'method': methods,
    'win round': winRounds,
    'win time': winTimes,
    'winner age': winnerAges,
    'loser age': loserAges,
    'winner wins': winnerWins,
    'winner loses': winnerLoses,
    'loser wins': loserWins,
    'loser loses': loserLoses,
    
    'winner height': winnerHeights,
    'loser height': loserHeights,
    'winner reach': winnerReachs,
    'loser reach': loserReachs,
    'winner leg reach': winnerLegReachs,
    'loser leg reach': loserLegReachs,
    
    'Event name': eventNames,
    'date': dates,
}

df = pd.DataFrame(data=dfData)

df.to_csv("my-data-set.csv");

df

In [ ]:
# creating the Dataframe
# df_winners_data = {
#     'Classs': "Winner",
#     'weight class': weightClasses,
#     'method': methods,
#     'win round': winRounds,
#     'win time': winTimes,
#     'age': winnerAges,
#     'wins': winnerWins,
#     'loses': winnerLoses,
#     'height': winnerHeights,
#     'reach': winnerReachs,
#     'leg reach': winnerLegReachs,
    
#     'Event name': eventNames,
#     'date': dates,
# }

# df_losers_data = {
#     'Classs': "Loser",
#     'weight class': weightClasses,
#     'method': methods,
#     'win round': winRounds,
#     'win time': winTimes,
#     'age': loserAges,
#     'wins': loserWins,
#     'loses': loserLoses,
#     'height': loserHeights,
#     'reach': loserReachs,
#     'leg reach': loserLegReachs,
    
#     'Event name': eventNames,
#     'date': dates,
# }

# df_winners = pd.DataFrame(data=df_winners_data)
# df_losers = pd.DataFrame(data=df_losers_data)

# all_data_df = pd.concat([df_winners, df_losers])

# all_data_df.to_csv("my-data-set.csv");

# all_data_df